# 데이터 불러오기 및 셋팅

In [27]:

# 필수 라이브러리
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential # 뼈대
from tensorflow.keras.layers import InputLayer,Dense,Activation,LSTM
from sklearn.metrics import r2_score
import seaborn as sns

In [18]:
# 전체 데이터셋 읽어들이기
raw_df = pd.read_csv('./data/continuous_factory_process.csv', index_col="time_stamp")
#1번 모두 처리함(5~14까지의 컬럼 제거 즉 10개.)
raw_df_modify = raw_df.drop(raw_df.filter(regex='Setpoint').columns,axis=1)

In [19]:
#제출데이터 만들기
submission_origin = pd.read_csv('./submission_data.csv', index_col="time_stamp")
#제출데이터 전처리
submission_data = submission_origin.drop(submission_origin.filter(regex='Setpoint').columns,axis=1)

In [20]:
#machin ambient~5,stage1,2,firstage 분리
raw_df_machine0 = raw_df_modify.filter(regex='AmbientConditions') 
raw_df_machine1 = raw_df_modify.filter(regex='Machine1') #machine1인 데이터값만 가져옴
raw_df_machine2 = raw_df_modify.filter(regex='Machine2') 
raw_df_machine3 = raw_df_modify.filter(regex='Machine3') 
raw_df_machine4 = raw_df_modify.filter(regex='Machine4') 
raw_df_machine5 = raw_df_modify.filter(regex='Machine5') 
raw_df_stage1 = raw_df_modify.filter(regex='Stage1') 
raw_df_stage2 = raw_df_modify.filter(regex='Stage2') 
raw_df_firststage = raw_df_modify.filter(regex='FirstStage') 

# stage1 문제


In [114]:
#stage1 x데이터 출력
X = pd.concat([raw_df_machine0,raw_df_machine1,raw_df_machine2,raw_df_machine3,raw_df_firststage],axis=1)

In [115]:
#stage1 y데이터 출력
y=raw_df_stage1

In [23]:
#stage1 데이터 전처리
#특정컬럼제거
#X=X.drop(['Machine2.RawMaterial.Property1','Machine2.RawMaterial.Property2','Machine3.MaterialTemperature.U.Actual'],axis=1)

In [116]:
#stage1 데이터 전처리
#스케일링
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(X)
df_sclaed_df = pd.DataFrame(df_scaled)
X = df_sclaed_df

In [117]:
#stage1 훈련 정답데이터 분리
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [118]:
#stage1 모델만들기
model_stage1 = Sequential()
model_stage1.add(LSTM(units=50,input_shape=(41,1)))
model_stage1.add(Dense(units = 128,activation='relu')) #활성화함수를 안에 같이 써줘도 되고 따로 써줘도 무방함.
model_stage1.add(Dense(units = 64,activation='relu'))
model_stage1.add(Dense(units = 32,activation='relu'))
model_stage1.add(Dense(units = 128,activation='relu'))
model_stage1.add(tf.keras.layers.Dropout(0.2)),
model_stage1.add(Dense(units=15))
model_stage1.summary()
model_stage1.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse'])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 50)                10400     
                                                                 
 dense_17 (Dense)            (None, 128)               6528      
                                                                 
 dense_18 (Dense)            (None, 64)                8256      
                                                                 
 dense_19 (Dense)            (None, 32)                2080      
                                                                 
 dense_20 (Dense)            (None, 128)               4224      
                                                                 
 dropout_3 (Dropout)         (None, 128)               0         
                                                                 
 dense_21 (Dense)            (None, 15)               

In [119]:
#stage1 모델 학습
model_stage1.fit(X_train,y_train,validation_split=0.2,  #훈련데이터 내에서 20% 검증데이터로 사용
          epochs=300)

Epoch 1/300
226/226 [==============================] - 5s 13ms/step - loss: 38.6679 - mse: 38.6679 - val_loss: 11.8307 - val_mse: 11.8307
Epoch 2/300
226/226 [==============================] - 3s 14ms/step - loss: 14.5155 - mse: 14.5155 - val_loss: 11.8551 - val_mse: 11.8551
Epoch 3/300
226/226 [==============================] - 3s 14ms/step - loss: 13.7762 - mse: 13.7762 - val_loss: 12.0665 - val_mse: 12.0665
Epoch 4/300
226/226 [==============================] - 3s 15ms/step - loss: 13.4694 - mse: 13.4694 - val_loss: 11.7857 - val_mse: 11.7857
Epoch 5/300
226/226 [==============================] - 4s 16ms/step - loss: 13.4096 - mse: 13.4096 - val_loss: 11.8270 - val_mse: 11.8270
Epoch 6/300
226/226 [==============================] - 4s 17ms/step - loss: 13.1931 - mse: 13.1931 - val_loss: 11.9742 - val_mse: 11.9742
Epoch 7/300
226/226 [==============================] - 3s 14ms/step - loss: 13.1192 - mse: 13.1192 - val_loss: 11.9012 - val_mse: 11.9012
Epoch 8/300
226/226 [=============

In [98]:
#stage1 모델 예측
model_stage1.predict(X_test) 

71/71 [==============================] - 0s 5ms/step


array([[13.044704  , 13.334288  , 11.286854  , ...,  1.5257133 ,
         3.1038427 , 14.693251  ],
       [12.850613  ,  0.65345573, 11.311343  , ...,  1.4228444 ,
         3.353666  ,  1.1356988 ],
       [12.850813  ,  9.800038  , 11.573832  , ...,  0.7363977 ,
         2.6759791 , 11.646754  ],
       ...,
       [13.040146  , 13.4546    , 11.273175  , ...,  1.5243726 ,
         3.101024  , 14.363474  ],
       [12.8120365 ,  7.9161134 , 11.600626  , ...,  0.7106029 ,
         2.722569  , 13.520613  ],
       [12.887492  ,  0.8125253 , 11.484982  , ...,  1.4860021 ,
         3.229618  , 11.463323  ]], dtype=float32)

In [110]:
#stage1 모델 정답데이터 만들기
#submission의 x데이터 가져오기
submission_stage_X = submission_data.iloc[:,0:41] 

#submission의 x데이터 전처리(모델에 학습했던 colunm selection 과 같게!)
#submission_stage_X=submission_stage_X.drop(['Machine2.RawMaterial.Property1','Machine2.RawMaterial.Property2','Machine3.MaterialTemperature.U.Actual'],axis=1)

#submission의 x데이터 전처리(stage1의 x데이터 처럼 스케일링!)
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(submission_stage_X)
df_sclaed_df = pd.DataFrame(df_scaled)
X_submission = df_sclaed_df
#stage1 모델 정답데이터 출력
model_submission1_pre = model_stage1.predict(X_submission)

89/89 [==============================] - 0s 5ms/step


In [112]:
#stage1 제출하기
#submission 데이터에 정답데이터 넣기
submission_data.iloc[:,42:57] =  model_submission1_pre
#submission1 저장하고 파일로 만들기
submission1 = submission_data.iloc[:,42:57]
np.save('submission1.npy', submission1)

,Stage1.Output.Measurement1.U.Actual,Stage1.Output.Measurement2.U.Actual,Stage1.Output.Measurement3.U.Actual,Stage1.Output.Measurement4.U.Actual,Stage1.Output.Measurement5.U.Actual,Stage1.Output.Measurement6.U.Actual,Stage1.Output.Measurement7.U.Actual,Stage1.Output.Measurement8.U.Actual,Stage1.Output.Measurement9.U.Actual,Stage1.Output.Measurement10.U.Actual,Stage1.Output.Measurement11.U.Actual,Stage1.Output.Measurement12.U.Actual,Stage1.Output.Measurement13.U.Actual,Stage1.Output.Measurement14.U.Actual,Machine4.Temperature1.C.Actual
time_stamp,,,,,,,,,,,,,,,
2019-03-06 10:52:34,12.881602,11.108870,11.525728,21.446243,31.974375,0.100744,2.394584,2.968608,21.012835,18.983297,7.777860,4.820940,0.772798,2.772620,14.534842
2019-03-06 10:52:35,12.864292,10.796167,11.518107,21.392389,31.814201,0.100739,2.360145,2.881801,20.888096,18.906410,7.756851,4.728960,0.779288,2.769287,14.227365
2019-03-06 10:52:36,12.848588,10.807688,11.494731,21.369123,31.795662,0.100813,2.359014,2.890857,20.840380,18.861372,7.753476,4.690063,0.785036,2.769865,14.270016
2019-03-06 10:52:37,12.870073,10.957961,11.514869,21.413769,31.891699,0.101447,2.372782,2.920685,20.953377,18.948387,7.767488,4.739079,0.782860,2.774833,14.384173
2019-03-06 10:52:39,12.590029,10.428984,11.209401,20.901497,30.987888,0.088306,2.202711,2.635536,19.195074,17.458477,7.608414,4.330048,0.759154,2.627956,13.159143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-06 14:46:57,12.849593,1.146642,11.325775,21.487192,33.346756,0.049396,0.332409,-0.012815,20.700445,18.849937,7.647273,0.140641,1.413460,3.295619,2.207462
2019-03-06 14:47:09,12.852317,1.153117,11.325607,21.496141,33.379215,0.048376,0.318794,-0.025988,20.714857,18.860575,7.652046,0.118026,1.415275,3.301226,2.097956
2019-03-06 14:47:14,12.850616,1.103623,11.322406,21.497688,33.391533,0.046241,0.308263,-0.021213,20.722038,18.868387,7.652266,0.108305,1.415003,3.306916,2.002542


# stage2


In [34]:
#stage2 x데이터 출력
X_stage2 = pd.concat([raw_df_machine0,raw_df_machine4,raw_df_machine5],axis=1)

In [80]:
#stage2 y데이터 출력
y_stage2 = raw_df_stage2

In [ ]:
#stage2 데이터 전처리
#특정컬럼제거

In [81]:
#stage2 데이터 전처리
df_scaled_stage2 = scaler.fit_transform(X_stage2)
df_sclaed_df_stage2 = pd.DataFrame(df_scaled_stage2)
X_stage2_scaler = df_sclaed_df_stage2

In [37]:
#stage2 훈련 정답데이터 분리
X_train_stage2,X_test_stage2,y_train_stage2,y_test_stage2 = train_test_split(X_stage2_scaler,y_stage2,test_size=0.2)

In [83]:
#stage2 모델만들기
model_stage2 = Sequential()
model_stage2.add(LSTM(units=20,input_shape=(16,1)))
model_stage2.add(Dense(units = 128,activation='relu')) #활성화함수를 안에 같이 써줘도 되고 따로 써줘도 무방함.
model_stage2.add(Dense(units = 64,activation='relu'))
model_stage2.add(Dense(units = 32,activation='relu'))
model_stage2.add(Dense(units = 16,activation='relu'))
model_stage2.add(Dense(units = 4,activation='relu'))
model_stage2.add(tf.keras.layers.Dropout(0.2)),
model_stage2.add(Dense(units=15))

model_stage2.summary()
model_stage2.compile(loss='mean_squared_error',optimizer='adam',metrics=['mse'])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 20)                1760      
                                                                 
 dense_11 (Dense)            (None, 128)               2688      
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 16)                528       
                                                                 
 dense_15 (Dense)            (None, 4)                 68        
                                                                 
 dropout_2 (Dropout)         (None, 4)                

In [84]:
#stage2 모델 학습
model_stage2.fit(X_train_stage2,y_train_stage2,validation_split=0.2,  #훈련데이터 내에서 20% 검증데이터로 사용
          epochs=300)

Epoch 1/300
226/226 [==============================] - 4s 7ms/step - loss: 56.9165 - mse: 56.9165 - val_loss: 23.3109 - val_mse: 23.3109
Epoch 2/300
226/226 [==============================] - 1s 4ms/step - loss: 29.0219 - mse: 29.0219 - val_loss: 16.0860 - val_mse: 16.0860
Epoch 3/300
226/226 [==============================] - 1s 5ms/step - loss: 24.5891 - mse: 24.5891 - val_loss: 14.0332 - val_mse: 14.0332
Epoch 4/300
226/226 [==============================] - 1s 5ms/step - loss: 23.4192 - mse: 23.4192 - val_loss: 14.3864 - val_mse: 14.3864
Epoch 5/300
226/226 [==============================] - 1s 5ms/step - loss: 22.3343 - mse: 22.3343 - val_loss: 13.6064 - val_mse: 13.6064
Epoch 6/300
226/226 [==============================] - 1s 5ms/step - loss: 21.8779 - mse: 21.8779 - val_loss: 13.9026 - val_mse: 13.9026
Epoch 7/300
226/226 [==============================] - 1s 5ms/step - loss: 22.0477 - mse: 22.0477 - val_loss: 14.0427 - val_mse: 14.0427
Epoch 8/300
226/226 [====================

In [85]:
#stage2 모델 예측
model_stage2.predict(X_test_stage2) 

71/71 [==============================] - 1s 3ms/step


array([[12.27515  ,  6.4108367, 10.771667 , ...,  2.0694313,  3.5150146,
         7.9315596],
       [12.005274 ,  6.2396727, 10.604524 , ...,  1.9900057,  3.7326427,
         7.4503784],
       [12.2745   ,  6.4067245, 10.765974 , ...,  2.069982 ,  3.517888 ,
         7.9219627],
       ...,
       [12.2551   ,  6.3526735, 10.694275 , ...,  2.0726485,  3.5600352,
         7.7921696],
       [12.266847 ,  6.358352 , 10.699018 , ...,  2.076461 ,  3.5516872,
         7.8090744],
       [12.273808 ,  6.4023476, 10.759915 , ...,  2.0705683,  3.5209463,
         7.911748 ]], dtype=float32)

In [86]:
#stage2 모델 정답데이터 만들기
#submission의 x데이터 가져오기
submission_stage2_X = submission_data.iloc[:,56:70] #stage2의 test 데이터(14개 특성)
#submission의 x데이터 전처리(모델에 학습했던 colunm selection 과 같게!)
#submission_stage2_X = submission_stage2_X.drop('Machine5.Temperature6.C.Actual',axis=1)
#submission의 x데이터 전처리(stage2의 x데이터 처럼 스케일링!)
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(submission_stage2_X)
df_sclaed_df = pd.DataFrame(df_scaled)
X_stage2_scaler = df_sclaed_df
#stage2 모델 정답데이터 출력
model_submission2_pre = model_stage2.predict(X_stage2_scaler) #stage2의 정답데이터

89/89 [==============================] - 0s 2ms/step


In [95]:
#stage2 제출하기
#submission 데이터에 정답데이터 넣기
#제출데이터 만들기
submission_origin = pd.read_csv('./submission_data.csv', index_col="time_stamp")#제출데이터 전처리
submission_data = submission_origin.drop(submission_origin.filter(regex='Setpoint').columns,axis=1)
submission_data.iloc[:,70:]  = model_submission2_pre
submission2 =submission_data.iloc[:,70:]
#submission2 저장하고 파일로 만들기
import numpy as np
np.save('submission2.npy', submission2)


,Stage2.Output.Measurement0.U.Actual,Stage2.Output.Measurement1.U.Actual,Stage2.Output.Measurement2.U.Actual,Stage2.Output.Measurement3.U.Actual,Stage2.Output.Measurement4.U.Actual,Stage2.Output.Measurement5.U.Actual,Stage2.Output.Measurement6.U.Actual,Stage2.Output.Measurement7.U.Actual,Stage2.Output.Measurement8.U.Actual,Stage2.Output.Measurement9.U.Actual,Stage2.Output.Measurement10.U.Actual,Stage2.Output.Measurement11.U.Actual,Stage2.Output.Measurement12.U.Actual,Stage2.Output.Measurement13.U.Actual,Stage2.Output.Measurement14.U.Actual
time_stamp,,,,,,,,,,,,,,,
2019-03-06 10:52:34,2.012209,1.992933,2.721025,11.143084,3.511558,1.850813,0.358587,1.444426,2.268785,1.695784,1.955846,1.365897,0.528392,2.534656,1.497746
2019-03-06 10:52:35,1.773598,1.896771,2.552872,11.003637,3.505978,1.838501,0.347992,1.411851,1.872971,0.944926,1.798144,1.248049,0.487334,2.524044,1.348482
2019-03-06 10:52:36,2.371701,2.138803,2.978165,11.366380,3.509716,1.871764,0.373193,1.494250,2.865499,2.703159,2.189040,1.539371,0.589084,2.554048,1.721280
2019-03-06 10:52:37,2.645916,2.233885,3.112266,11.321142,3.675618,1.848543,0.406465,1.520062,3.314453,5.493536,2.438739,1.738237,0.654397,2.513266,1.913810
2019-03-06 10:52:39,0.505266,1.358698,1.555845,9.903786,3.754661,1.707840,0.328492,1.218413,-0.241301,0.317639,1.079374,0.732337,0.300734,2.375179,0.591704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-06 14:46:57,11.907097,6.217096,10.600464,20.074972,1.298725,2.934066,0.474637,2.973407,18.773403,3.293509,7.446248,5.280759,1.953149,3.786615,7.365846
2019-03-06 14:47:09,11.915162,6.218951,10.600798,20.061096,1.313342,2.931102,0.476904,2.973456,18.786247,3.493259,7.457773,5.290481,1.956176,3.782181,7.372791
2019-03-06 14:47:14,11.915539,6.219038,10.600813,20.060448,1.314025,2.930963,0.477010,2.973459,18.786846,3.502595,7.458311,5.290936,1.956318,3.781974,7.373115
